In [1]:
import sys
# To import python scripts from other folders
sys.path.append('../')
import matplotlib.pyplot as plt
import json
from ProjectTestAnalysis import ProjectTestAnalysis
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import re
import concurrent
from statistics import median, mean
import numpy as np
root="/home/jovyan/work"
results_path=root+"/results/"
procesed_results_path=root+"/notebooks/ProjectAnalysis/TestAnalysis/results/"

In [2]:
def getProjects(dataset):
    path = root+"/configFiles/%sProjects/"%dataset
    projects = []
    for configFile in os.listdir(path):
        with open(path+configFile) as f:
            project_info = json.load(f)
            if os.path.isdir(procesed_results_path+project_info["project"]):
                projects.append((dataset, project_info["project"]))
    return projects

In [3]:
# Get Projects names
github = getProjects("GitHub")
apache = getProjects("Apache")
many4j = getProjects("ManySStub4J")
all_datasets = github + apache + many4j

In [4]:
all_projects_src_errors = []
all_projects_test_errors = []

total = 0

for dataset, project in all_datasets:
    
    build_test_errors_path = procesed_results_path+project+"/build-test-errors.json"
    
    
    if os.path.isfile(build_test_errors_path):
        
        pa = ProjectTestAnalysis(project,  2, root=root)
        report_df = pa.getReport()
        
        with open(build_test_errors_path) as f:
            errors = json.load(f)
            
        for _, commit in report_df.iterrows():
            c_hash = commit['commit']
            c_errors = errors[c_hash]
            
            if commit['build'] == 'SUCCESS':
                if commit['test_build'] == 'SUCCESS':
                    pass # DO NOTHING - TEST COULD BE EXECUTED
                else:
                    test_build_main_cause = c_errors['test_build']["main_cause"]
                    all_projects_test_errors.append(test_build_main_cause)
            else:
                src_build_main_cause = c_errors['src_build']["main_cause"]
                all_projects_src_errors.append(src_build_main_cause)

In [5]:
src_errors_df = pd.DataFrame(all_projects_src_errors, columns =['Cause'])
src_errors_df.groupby(['Cause']).size()

Cause
Compilation         5565
NO BUILD SYSTEM    42585
Other              39395
Parsing                1
Resolution         31057
dtype: int64

In [6]:
test_errors_df = pd.DataFrame(all_projects_test_errors, columns =['Cause'])
test_errors_df.groupby(['Cause']).size()

Cause
Other         2643
Resolution     202
dtype: int64